In [1]:
from quchem_ibm.Qiskit_Chemistry import *

In [2]:
transformation='BK'

In [3]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
## HAMILTONIAN start

Molecule = 'H2'
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation=transformation)
### HAMILTONIAN end

#####################################

print(QubitHamiltonian)


fci_energy = Hamilt.molecule.fci_energy
print(fci_energy)

(-0.09706626816762878+0j) [] +
(0.045302615503799264+0j) [X0 Z1 X2] +
(0.045302615503799264+0j) [X0 Z1 X2 Z3] +
(0.045302615503799264+0j) [Y0 Z1 Y2] +
(0.045302615503799264+0j) [Y0 Z1 Y2 Z3] +
(0.1714128264477689+0j) [Z0] +
(0.17141282644776892+0j) [Z0 Z1] +
(0.1659278503377034+0j) [Z0 Z1 Z2] +
(0.1659278503377034+0j) [Z0 Z1 Z2 Z3] +
(0.12062523483390414+0j) [Z0 Z2] +
(0.12062523483390414+0j) [Z0 Z2 Z3] +
(0.16868898170361207+0j) [Z1] +
(-0.22343153690813558+0j) [Z1 Z2 Z3] +
(0.1744128761226159+0j) [Z1 Z3] +
(-0.22343153690813564+0j) [Z2]
-1.137283834488502


From Helgaker, T., P. Jorgensen,and J. Olsen (2014),Molecularelectronic-structure theory(John Wiley & Sons)
we known

$$H_{2}^{ground} = 0.9939| 1100\rangle - 0.1106| 0011\rangle$$

From group theory we know only double excitation terms are important!
We can see this from our ijab operators that qubits 1 and 3 are only acted on by $Z$ therefore experience no population changes... Can even see this from the answer if we write it in BK basis:

$$H_{2}^{BK} = 0.9939| 1000\rangle - 0.1106| 0010\rangle$$

See how qubits 1 and 3 always remain at 0!

therefore can **remove** these measurements from Hamiltonian... as we know what expectation values they should give!

In the case for this Hamiltonian only $I$ and $Z$ act on these qubits! So even easier to remove (expectation values = +1 (as act on $| 0\rangle$ state!), so just remove and add to relivent H terms!

LOOK at:

PHYS. REV. X, **8**, 031022 (2018)

In [4]:
from quchem.Ansatz_Generator_Functions import *
n_electrons=Hamilt.molecule.n_electrons
n_qubits=Hamilt.molecule.n_qubits

ansatz_obj = Ansatz(n_electrons,n_qubits)
print('JW ground state = ', ansatz_obj.Get_JW_HF_state_in_OCC_basis())
print('BK ground state = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

JW ground state =  [1, 1, 0, 0]
BK ground state =  [1. 0. 0. 0.]


In [5]:
qubits_to_remove = Find_I_Z_indices_in_Hamiltonian(QubitHamiltonian, Hamilt.molecule.n_qubits)
print('qubits only acted on by I or Z:', qubits_to_remove)


input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# Remove_Z_terms_from_Hamiltonian
NewQubitHamiltonian = Remove_Z_terms_from_Hamiltonian(
                                                    QubitHamiltonian, 
                                                    input_state,
                                                    qubits_to_remove,
                                                    check_reduction=True)
NewQubitHamiltonian

qubits only acted on by I or Z: [1 3]


(0.2460355896585992+0j) [] +
(0.09060523100759853+0j) [X0 X2] +
(0.09060523100759853+0j) [Y0 Y2] +
(0.3428256528955378+0j) [Z0] +
(0.5731061703432151+0j) [Z0 Z2] +
(-0.4468630738162712+0j) [Z2]

In [6]:
qubitNo_re_label_dict, NewQubitHamiltonian_relabelled = Re_label_Hamiltonian(NewQubitHamiltonian)
NewQubitHamiltonian_relabelled

(0.2460355896585992+0j) [] +
(0.09060523100759853+0j) [X0 X1] +
(0.09060523100759853+0j) [Y0 Y1] +
(0.3428256528955378+0j) [Z0] +
(0.5731061703432151+0j) [Z0 Z1] +
(-0.4468630738162712+0j) [Z1]

# Find what new FCI energy is
- here should be the same as true answer!

In [7]:
from openfermion import qubit_operator_sparse
from scipy.sparse import csc_matrix
new_Molecular_H_MATRIX =  csc_matrix(qubit_operator_sparse(NewQubitHamiltonian_relabelled))

# new_Molecular_H_MATRIX = np.flip(new_Molecular_H_MATRIX)

from scipy.sparse.linalg import eigs
try:
    eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
except:
    from scipy.linalg import eig
    eig_values, eig_vectors = eig(new_Molecular_H_MATRIX.todense())
    
new_FCI_Energy = min(eig_values)


index = np.where(eig_values==new_FCI_Energy)[0][0]
ground_state_vector = eig_vectors[:, index]

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', fci_energy)
print(np.isclose(new_FCI_Energy, fci_energy))

new_FCI =  (-1.1372838344885006+0j) VS old FCI: -1.137283834488502
True


/home/lex/anaconda3/envs/QuantumUpdated/lib/python3.7/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1269: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


In [8]:
np.dot(ground_state_vector.conj().T, new_Molecular_H_MATRIX.dot(ground_state_vector))

(-1.137283834488501+0j)

# HF + UCCSD

In [9]:
input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
new_input_state = np.delete(input_state, qubits_to_remove)

n_qubits_new = len(new_input_state)
new_input_state

array([1., 0.])

In [10]:
# ansatz_obj.Get_ia_and_ijab_terms()

# print('ia standard')
# print(ansatz_obj.Sec_Quant_CC_ia_Fermi_ops)

# print('ijab standard')
# print(ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

# ### single trotter step
# ansatz_obj.UCCSD_single_trotter_step(transformation,
#                                      List_FermiOps_ia=ansatz_obj.Sec_Quant_CC_ia_Fermi_ops,
#                                      List_FermiOps_ijab=ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

# print('')
# print('ia standard')
# print(ansatz_obj.Second_Quant_CC_single_Trot_list_ia)
# print('ijab standard')
# print(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab)

In [11]:
# new_CC_ia_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ia,
#                                                       qubits_to_remove)
# print('ia reduced')
# print(new_CC_ia_single_trot)

# new_CC_ijab_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab,
#                                                       qubits_to_remove)
# print('ijab reduced')
# print(new_CC_ijab_single_trot)


# relabelled_new_CC_ia_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ia_single_trot)
# relabelled_new_CC_ijab_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ijab_single_trot)

# print('')
# print('ia reduced relabelled')
# print(relabelled_new_CC_ia_single_trot)
# print('')
# print('ijab reduced relabelled')
# print(relabelled_new_CC_ijab_single_trot)

# Ansatz Circuit

In [12]:
# for op in relabelled_new_CC_ia_single_trot[1]:
#     print(op)
# print(list(op.terms.keys())[0])

In [13]:
# UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(new_input_state,
#                                      relabelled_new_CC_ia_single_trot, 
#                                      relabelled_new_CC_ijab_single_trot)

# theta_ia = [0 for _ in range(len(relabelled_new_CC_ia_single_trot))]
# theta_ijab = [0 for _ in range(len(relabelled_new_CC_ijab_single_trot))]

# UCCSD_ansatz_Q_Circ =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
#                                         Theta_param_list_ia=theta_ia, 
#                                          Theta_param_list_ijab=theta_ijab,
#                                          ia_first=True)
# UCCSD_ansatz_Q_Circ

In [14]:
n_qubits= len(new_input_state)
check_ansatz_state = False
decompose_fully=True

Ansatz_circuit, q_reg = Vector_defined_Ansatz(n_qubits,
                                              ground_state_vector,
                                              check_ansatz_state=check_ansatz_state,
                                              decompose_fully=decompose_fully)

# Get standard VQE circuits

In [16]:
standard_VQE_circuits, standard_I_term = Build_Standard_VQE_circuits(
                                                    NewQubitHamiltonian_relabelled, 
                                                    Ansatz_circuit,
                                                    q_reg)

Getting_standard_VQE_circuits: 6it [00:00, 3791.75it/s]


In [17]:
# lin alg approach!
standard_VQE_lin_alg(NewQubitHamiltonian_relabelled,
                     ground_state_vector,
                     n_qubits,
                     check_ansatz_state=check_ansatz_state)

performing_standard_VQE: 6it [00:00, 1846.22it/s]


(-1.1372838344885015+0j)

# Graph

In [18]:
from tqdm.notebook import tqdm

In [19]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(NewQubitHamiltonian_relabelled)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)

anti_commuting_sets

Building Graph Edges: 100%|##########| 6/6 [00:00<00:00, 4401.93it/s]


{0: [(0.5731061703432151+0j) [Z0 Z1]],
 1: [(0.2460355896585992+0j) []],
 2: [(-0.4468630738162712+0j) [Z1], (0.09060523100759853+0j) [X0 X1]],
 3: [(0.3428256528955378+0j) [Z0], (0.09060523100759853+0j) [Y0 Y1]]}

# Seq Rot circuits

In [20]:
n_qubits= len(new_input_state)
rotation_reduction_check=True

Seq_Rot_VQE_circuits, Seq_Rot_I_term = Get_Seq_Rot_Unitary_Part_circuits(
                                anti_commuting_sets, 
                                Ansatz_circuit,
                                q_reg,
                                n_qubits,
                                S_index_dict=None, 
                                rotation_reduction_check=rotation_reduction_check)

Getting seq_rot VQE circuits: 100%|##########| 4/4 [00:00<00:00, 399.96it/s]


In [21]:
Seq_Rot_VQE_circuits

[{'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7efd48052790>,
  'coeff': (0.5731061703432151+0j),
  'qubitOp': (0.5731061703432151+0j) [Z0 Z1]},
 {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7efd48052710>,
  'gamma_l': (0.455956044621043+0j),
  'Ps': 1 [Z1]},
 {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7efd48052ad0>,
  'gamma_l': (0.35459658228639496+0j),
  'Ps': 1 [Z0]}]

In [22]:
#lin alg approach

n_qubits= len(new_input_state)
check_ansatz_state = False
rotation_reduction_check=True

Seq_Rot_VQE_lin_alg(anti_commuting_sets,
                    ground_state_vector,
                    n_qubits,
                    S_index_dict=None,
                    rotation_reduction_check=rotation_reduction_check,
                     check_ansatz_state=check_ansatz_state)

Performing seq_rot VQE lin alg: 100%|##########| 4/4 [00:00<00:00, 321.35it/s]


(-1.137283834488501+0j)

# LCU circuits

In [23]:
n_qubits= len(new_input_state)

LCU_VQE_circuits, LCU_I_term= Get_LCU_Unitary_Part_circuits(anti_commuting_sets, 
                                                            Ansatz_circuit, 
                                                            q_reg,
                                                            n_qubits, 
                                                            N_index_dict=None)

Getting LCU VQE circuits: 100%|##########| 4/4 [00:00<00:00, 92.27it/s]


In [24]:
LCU_VQE_circuits

[{'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7efd48046910>,
  'coeff': (0.5731061703432151+0j),
  'qubitOp': (0.5731061703432151+0j) [Z0 Z1]},
 {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7efd35fbad10>,
  'gamma_l': (0.455956044621043+0j),
  'Pn': 1 [Z1],
  'N_ancilla': 1},
 {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7efd35fdc650>,
  'gamma_l': (0.35459658228639496+0j),
  'Pn': 1 [Z0],
  'N_ancilla': 1}]

In [25]:
#lin alg approach
n_qubits= len(new_input_state)
check_ansatz_state = False

LCU_VQE_lin_alg(anti_commuting_sets,
                ground_state_vector,
                n_qubits,
                N_index_dict=None,
             check_ansatz_state=check_ansatz_state)

Performing LCU VQE lin alg: 100%|##########| 4/4 [00:00<00:00, 41.91it/s]


(-1.1372838344885015+1.7869758757458474e-18j)

# Save experiment input

In [26]:
list(NewQubitHamiltonian_relabelled)

[(0.2460355896585992+0j) [],
 (0.3428256528955378+0j) [Z0],
 (-0.4468630738162712+0j) [Z1],
 (0.09060523100759853+0j) [Y0 Y1],
 (0.09060523100759853+0j) [X0 X1],
 (0.5731061703432151+0j) [Z0 Z1]]

In [27]:
filename = 'H2_bravyi_kitaev_2_qubit_experiment'
n_qubits= len(new_input_state)

Save_exp_inputs(filename, NewQubitHamiltonian_relabelled, anti_commuting_sets, Hamilt.geometry, basis, transformation,
                    Graph_colouring_strategy, fci_energy,
                    standard_VQE_circuits, standard_I_term,
                    Seq_Rot_VQE_circuits, Seq_Rot_I_term,
                    LCU_VQE_circuits, LCU_I_term,
                    ground_state_vector,
                    n_qubits,
                    S_index_dict=None,
                    N_index_dict=None)

experiment data saved here: /home/lex/Documents/PhD/VQE-code/quchem_ibm/Experiments/Input_data/H2_bravyi_kitaev_2_qubit_experiment_time=2020Sep21-162239536536
